## Contents <a id='top'></a>

1. <a href=#np>NumPy</a>
1. <a href=#pandas>Pandas</a>
    1. <a href=#pandas>Series</a>
    1. <a href=#df>DataFrames</a>
    1. <a href=#functions>Applying Functions to DataFrame</a>
    1. <a href=#categorical>Categorical Variables</a>
    1. <a href=#split>Split-Apply-Transform</a>
    1. <a href=#multiple>Working with Multiple DataFrames</a>
1. <a href=#ref>References and Links</a>

In [ ]:
import numpy as np
import pandas as pd

# %precision 4
# %matplotlib inline

# pd.set_option('display.precision', 4)

<a id='np'></a>
# 1. NumPy
<a href=#top>(back to top)</a>

The basic object in this package is the `ndarray` object, which can represent *n*-dimensional arrays of homogeneous data types. This is the key difference between NumPy and Pandas objects -- while Pandas objects are also tabular in nature, they allow you to deal with inhomogenous objects. Specifically, Pandas' `DataFrame`s allow columns to be of different types.

An `ndarray` object is an *n*-dimensional array (i.e., a [tensor](https://www.i2tutorials.com/wp-content/uploads/2019/09/Tensor-Datatype-and-Ranks-1i2tutorials.jpg)) of elements, indexed by a tuple of non-negative integers.

The dimensions of the array are referred to as **axes** in NumPy: a three-dimensional array will have three axes.

Each array has several attributes. These include:
* `ndim`: the number of axes/dimensions.
* `shape`: a tuple describing the length of each dimension.
* `size`: the total number of elements in the array. This is a product of the integers in the shape attribute.

In [ ]:
arr = np.array([(1.5, 2, 3), (4, 5, 6)])

In [ ]:
arr

In [ ]:
arr.ndim

In [ ]:
arr.shape

In [ ]:
arr.size # the number of elements in "arr"

### Array Creation

One way to create an array containing regular sequences is to use the `np.arange()` function. This creates a sequence of integers, with a specified separation.

In [ ]:
seq = np.arange(0, 10, 3)
seq

The shape of an `ndarray` is given by a tuple. Note that an array of shape (4,) is different from one with shape (4, 1). The former has only **1 dimension**, while the latter has **2 dimensions**.

In [ ]:
seq.shape

In [ ]:
col_vect = seq.reshape(4, 1)
col_vect

To create an array of regularly spaced **real numbers**, use `np.linspace()`.

In [ ]:
arr_real = np.linspace(start = 0.2, stop = 3.3, num = 24).reshape(2, 3, 4)  
arr_real

In [ ]:
np.arange(24).reshape(4, 3, 2)

Sometimes we need to create a placeholder array with the appropriate dimensions, and then fill it in later. This is preferrable to growing an array by appending to it.

In [ ]:
np.zeros((3, 5)) # there is also an np.ones() function

Instead of specifying the dimensions of an array ourselves, we can create arrays of zeros or ones in the shape of other existing arrays.

In [ ]:
# Creates an array of zeros, of the same shape as "arr_real".
np.ones_like(arr_real)

### Slice Operator in Multiple Dimensions

Multidimensional NumPy arrays can be accessed with comma separated slice notation. When fewer indices are provided than the number of axes, the missing indices are considered complete slices for the remaining dimensions.

By the way, when printing, the *last* axis will be printed left-to-right, and the second last axis will be printed from top-to-bottom. The remaining axes will be printed with a line in between:

In [ ]:
arr_real

Here are examples based on this array. Try to guess what each will return before you run it:

In [ ]:
arr_real[1, 2, 3]

In [ ]:
arr_real[0, 2, ::-1]

In [ ]:
arr_real[1, 0:3:2]

In [ ]:
arr_real[:, 2, :]

Here are examples using Boolean indexing, which means that we use an array of `True` and `False` entries to determine which elements to return.

In [ ]:
arr_real > 3

In [ ]:
arr_real[arr_real > 3]

### Basic Operations

In [ ]:
# Setting a seed allows for reproducibility of random number generation
# across sessions.
np.random.seed(5003)

In [ ]:
a = np.random.randn(3, 5)
a

In [ ]:
b = np.random.randn(3, 5)
b

In [ ]:
# Element-wise addition.
a + b

In [ ]:
# Element-wise multiplication: NOT matrix multiplication.
a * b

In [ ]:
# Matrix multiplication (need to transpose "b" to match get the right dimensions).
# We can also do "a @ b.T".
a.dot(b.T)

### Broadcasting

In [ ]:
2 * a + 1

It's easy enough to guess what happened there, but what if we added a 4 x 5 array to a 2 x 1 x 5 array? In general what would happen if we operated on two arrays with different shapes? 

That's when the broadcasting rules come into play.

1. If all input arrays do not have the same number of dimensions, a "1" will be *pre-pended* to the **shapes** of the smaller arrays until all the arrays have the same number of dimensions.
2. Arrays with size 1 along a particular dimension act as if they had the size of the array with the largest shape along that dimension.

In [ ]:
x = np.arange(4)           # shape           4
x

In [ ]:
xx = x.reshape((4, 1))      # shape       4 x 1
xx

In [ ]:
xx + x                     # shape       4 x 4

`x` acts like an array with size (1, 4) in the addition above, and rule 2 applies for `xx`.
The following code gives us the same output:

In [ ]:
np.tile(xx, 4) + x.reshape((1, 4))

In [ ]:
y = np.ones(5)           # shape           5
x + y

In [ ]:
xx + y                  # shape        4 x 5

In [ ]:
z = np.ones((3, 4))
x + z

### Axis-wise Operations

In [ ]:
arr_real.shape

In [ ]:
arr_real

In [ ]:
arr_real.mean(axis = 0) # mean across the 0th ("first") axis

The top-left element comes from the mean of `arr_real[0, 0, 0]` and `arr_real[1, 0, 0]`. Similarly, the element to the right of it comes from the mean of `arr_real[0, 0, 1]` and `arr_real[1, 0, 1]`.

In [ ]:
(arr_real[0, 0, 1] + arr_real[1, 0, 1]) / 2

In [ ]:
arr_real.mean(axis = 1)

In [ ]:
arr_real.mean(axis = (0, 1)) # the mean across the first two axes

In [ ]:
arr_real.mean(axis = 2) # which is the mean across the third axis

Note that `arr_real[0]` is a 2D array, with shape (3, 4). Suppose we wish to compute the row means. This means we have to apply the operation by the column axis (axis = 1).

In [ ]:
arr_real[0].mean(axis = 1) # the mean across the second axis of arr_real[0], not of arr_real itself.

If we wanted to identify the row with the largest mean, we use `argmax()` on the resulting array.

In [ ]:
arr_real[0].mean(axis = 1).argmax()

<a id='pandas'></a>
# 2. Pandas
<a href=#top>(back to top)</a>

## Series 

A *Series* is a one-dimensional labeled array. The axis labels are referred to as the **index**. The simplest way to create a Series is to pass a sequence and an index to `pd.Series()`.

In [ ]:
year = pd.Series(list(range(2010, 2013) ) * 3)

team = ["Barcelona", "RealMadrid", "Valencia"] * 3
team.sort()
team = pd.Series(team)

wins = pd.Series([30, 28, 32, 29, 32, 26, 21, 17, 19])
draws = pd.Series([6, 7, 4, 5, 4, 7, 8, 10, 8])
losses = pd.Series([2, 3, 2, 4, 2, 5, 9, 11, 11])

In [ ]:
wins.index

In [ ]:
wins.values

To access particular values, we can use the slice operator.

In [ ]:
wins[0:6:2]

To convert a Series object to an `ndarray`, we use the following method:

In [ ]:
wins.to_numpy()

If we specify an index, we can use it to access values in the Series. With Pandas, using the slice operator with labels is inclusive on both sides!

In [ ]:
s = pd.Series(np.random.randn(5), 
             index=['a', 'b', 'c', 'd', 'e'])
s

In [ ]:
s.index

In [ ]:
s.values

In [ ]:
s[['a', 'c']] 

Be careful when you combine the slice operator with label-based indexing. Unlike vanilla Python, Pandas includes **both** end-points!

In [ ]:
s['a':'d']

<a id='df'></a>
## DataFrames

A *DataFrame* is a 2-dimensional labeled data structure with possibly **different** data types. It is the most commonly used Pandas object. The *index* of a DataFrame refers to the row labels (axis 0). The *columns* refer to the column labels (axis 1).

DataFrames can be constructed from Series, dictionaries, lists and 2-d arrays. For our course, we will typically create a DataFrame directly from a file.

We can create a DataFrame from the earlier series.

In [ ]:
laliga = pd.DataFrame({'Year': year,
              'Team': team,
              'Wins': wins,
              'Draws': draws,
              'Losses': losses
})

To inspect a DataFrame, we can use `info()`, `head()` and `tail()` methods.

In [ ]:
laliga.info()

In [ ]:
laliga.head()

We can also export the DataFrame to disk as plaintext (e.g., JSON, CSV, etc.).

In [ ]:
laliga.to_json('../data/laliga.json')

### DataFrame from NumPy Arrays

In [ ]:
np.random.seed(5003)
x = np.random.randn(12, 5)
x_df = pd.DataFrame(x, columns = list('ABCDE')) # list(string) creates a list of its characters.

In [ ]:
x_df.head(n = 3)

In [ ]:
# We can convert DataFrames back to ndarrays.
x_df.to_numpy() == x

### Reading in Data

The CSV file read here contains the happiness scores of 164 countries from 2015 to 2017. Click [here](https://worldhappiness.report/ed/2017/) for a full report on the dataset.

The final score was based on many other factors (such as GDP per capita, family, freedom etc) which is included in the file as well. We will simplify things by just reading in the country, final score computed and year.

In each year, not all of the 164 countries had their scores surveyed and taken. This results in some countries having missing values (`NaN`) in certain years.

Besides reading in CSV formats, pandas can read in tab-separated files, Excel files and HDF5 files.

In [ ]:
happ = pd.read_csv('../data/happiness_report.csv', header = 0, na_values = 'NA')

happ.head()
#happ.tail()
#happ.info()

In [ ]:
happ.columns

In [ ]:
happ.shape

In [ ]:
happ.describe()

In [ ]:
happ['Govt.Corruption'].plot(kind = "hist");

### Basic Selection

Row selection can be done with integers in the slice operator. In practice, this is not used, because we typically wish to select a set of rows based on a condition.

In [ ]:
happ[10:12]

To select columns, you may use a list of column names.

In [ ]:
happ[['GDP', 'Freedom']] # note the difference with happ['GDP']
# happ.GDP.head()

Remember that we are not working with numpy arrays, so this will not work:

In [ ]:
happ[0:10, 2:4]

### Indexing and Selecting Data

The two main methods of advanced data selection use the `.loc` and `.iloc` functions. Although we call them functions, they are summoned using the `[ ]` notation. The `.loc` is primarily label-based. The common allowed inputs to `.loc` are 
* a single label,
* a list of labels,
* a slice object, 
* a boolean array.

The `.iloc` is primarily an integer-based input. The common allowed inputs to `.iloc` are 
* a single integer,
* a list of integers,
* a slice object, 
* a boolean array.
  
When selecting from a DataFrame with `.loc` or `.iloc`, we can provide a comma-separated index, just as with NumPy. It is good to keep this [reference](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) page bookmarked.

*Take note that this next command will only work if the Index is made up of integers!*

In [ ]:
happ.loc[2:5]

In [ ]:
happ.loc[[2, 3, 4, 5]]

Notice how the slice operator inclusive when we use `.loc`, but not inclusive when we use `.iloc`.

In [ ]:
happ.loc[2:10:4, "GDP":"Generosity":2]

In [ ]:
happ.iloc[2:11:4, 3:8:2] # Same as above, but with .iloc

### Filtering Data

In [ ]:
happ.iloc[:, 2:4].describe(percentiles=[0.25, .5, .75, .9])

It looks like Happiness Score is a left-skewed distribution. Suppose we are interested in the very happy countries. Here is how we can filter the data with a boolean array.

In [ ]:
happiest = happ[happ['Happiness.Score'] > 6.95]

In [ ]:
happiest.Country.unique()

There isn't a single Asian or African country in the happiest 10% of countries!

We can also combine Boolean indices.

In [ ]:
# Top 3 happiest countries in 2015
happ[(happ.Year == 2015) & (happ['Happiness.Rank'] <= 3)]

### Missing Values

The `.info()` method will yield information on missing values, column by column. We can see there are 21 rows with missing values. 

In [ ]:
happ.info()

*Sometimes*, it is appropriate to drop rows with missing values. This can be done with the `.dropna` method. Remember that it returns a new dataframe. The original one remains unchanged, unless you include the `inplace=True` argument.

In [ ]:
happ.dropna()

In [ ]:
pd.isna(happ)

<a id='functions'></a>
## Applying Functions to DataFrame

A common task in data analysis is to apply a function to each column, or to each row. With a Pandas DataFrame, this can be achieved in a few ways, depending on how the function is to be applied:
* To apply a function to the entire DataFrame, e.g. fitting a model, we use the `pipe` function. We shall see this later in the course.
* To apply a function to each row or each column, we use the `apply` function.
* To apply a function to each element, we use the `applymap` function. This is typically used with a lambda function.
* There are several built-in functions that we can call to aggregate a column or a row. Examples of these are `count`, `sum`, `mean`, `median`, and so on. A full list can be found [here](https://pandas.pydata.org/docs/user_guide/basics.html#descriptive-statistics).

Remember that the happiness dataframe contains 11 columns, but not all of them are numerical measures. Suppose we wish to count how many non-null values there are in each of the columns, apart from Happiness Rank, Country, and Year.

In [ ]:
happ.loc[:, 'Happiness.Score':'Dystopia.Residual'].count(axis = 0)

The `count()` function was applied to each column (along `axis=0`). Similarly, suppose we wish to compute the standard deviation of each column.

In [ ]:
happ.loc[:, 'Happiness.Score':'Dystopia.Residual'].std(axis = 0)

Suppose we wish to find the **range** of values for each numeric column, which we define as the maximum minus the minimum value of that column. We can apply a lambda function of our own here.

In [ ]:
pd.DataFrame.apply?

In [ ]:
happ.loc[:, 'Happiness.Score':'Dystopia.Residual'].apply(lambda x: x.max() - x.min())

Between-Series operations are conducted element-wise. Note the two different ways of indexing a column.

In [ ]:
happ['Life.Expectancy'] / happ.Freedom

### Adding or Deleting Columns

To delete columns, we can use the DataFrame `.drop()` method. Unless we specify `inplace=True`, a new copy of the DataFrame will be created by `.drop()`.

In [ ]:
# To delete columns:
happ.drop(columns=['GDP', 'Freedom', 'Year']).head(n = 3)

We can create a new column simply by specifying it.

In [ ]:
# To create a new column, that contains square root of Happiness Score.
happ['sqrt_HS'] = np.sqrt(happ['Happiness.Score'])
#happ.loc[:, 'sqrt_HS'] = happ['Happiness.Score'].apply(np.sqrt)

Suppose we were given the following classification of GDP. We can create that new feature 
as follows.

In [ ]:
happ.GDP.describe()

In [ ]:
happ.GDP.plot(kind = "hist");

In [ ]:
# To create a new column, that bucketizes the GDP into high, low, medium
happ['GDP_cat'] = pd.cut(happ.GDP, bins = [0, 0.5, 1.5, 1.870765686], labels = ['low', 'med', 'high'] )
#happ.loc[:, 'GDP_cat'] = pd.cut(happ.GDP, bins = [0, 0.5, 1.5, 2], labels = ['low', 'med', 'high'] )

In [ ]:
happ.loc[0:5, ['GDP', 'GDP_cat']]

<a id='categorical'></a>
## Categorical Variables

### Tables of Counts

To summarise categorical variables, one option is to use tables. These tables could present counts of levels, or proportions.

When assessing the relationship between categorical variables, the simplest option is to print a table of counts.

When we wish to assess a variable, or an aggregation of it, conditional on two or more categorical levels, we use a pivot table.

Suppose that, in the happiness dataset, we wish to understand if the distribution of high, low, med GDP has changed in the three years. We can set up a cross-table.

In [ ]:
pd.crosstab(happ.Year, happ.GDP_cat)

It is interesting to note that in 2017, the tails of the distribution changed the most. We can compute the proportion of high/med/low within each year by adding the normalize argument. What else piques your curiosity?

In [ ]:
tab_out = pd.crosstab(happ.Year, happ.GDP_cat, normalize = 'index') 
tab_out

In [ ]:
tab_out.plot(kind = 'bar', figsize = (10, 4));

Does Happiness Score change with GDP category (as we have defined them) over the years? To introduce a third variable, we shall have to create a pivot table.

In [ ]:
happ.pivot_table(index = 'Year', columns = 'GDP_cat', values = 'Happiness.Score', aggfunc = 'mean')

To complicate things further, suppose we compare SEA nations to non-SEA nations.

In [ ]:
SEA_nations = ['Singapore', 'Malaysia', 'Indonesia', 'Laos', 'Thailand', 'Vietnam', 'Myanmar']
happ['region'] = happ.Country.apply(lambda x: 'SEA' if x in SEA_nations else 'not_SEA')

In [ ]:
happ

In [ ]:
happ.pivot_table(index = ['region', 'Year'], columns = 'GDP_cat', values = 'Happiness.Score', aggfunc = 'mean')

<a id='split'></a>
## Split-Apply-Transform

### Grouping

In [ ]:
happ.groupby(['Country', 'Year'])['Happiness.Score'].mean().sort_values()

In [ ]:
happ.groupby('Country')['Happiness.Score'].mean().sort_values(ascending = False).head()

### Basic Plotting

In [ ]:
happ.head()

In [ ]:
happ.describe()

In [ ]:
happ.plot(x = 'Generosity', y = 'Life.Expectancy', kind = 'scatter');

In [ ]:
happ[happ.Generosity > 0.55]

In [ ]:
happ.Generosity.plot(kind = 'box');

<a id='multiple'></a>
##  Working with Multiple DataFrames

The following two datasets contain information on the matches played at the 2018 FIFA world cup:

* `match_schedule.xlsx` - contains the schedule of all 64 matches and their outcomes.
* `weatherTable.xlsx` - contains information on the weather at the venue of each match.

In [ ]:
weather = pd.read_excel('../data/weatherTable.xlsx')
schedule = pd.read_excel('../data/match_schedule.xlsx')
# You may need to install openpyxl (at command prompt, run pip install openpyxl).

In [ ]:
weather.head()

In [ ]:
schedule.head()

Both dataframes have a match id column. When we want to merge two dataframes, they have to have common column(s). There are two main types of joins: inner and outer joins. Outer joins are left, right or full joins. 

<img src="../figs/join-outer.png" style="width: 450px;"/>

Inner joins keep only the rows that are present in **both** dataframes.

<img src="../figs/join-inner.png" style="width: 450px;"/>

With pandas, things  get an additional layer of complication. Instead of merging on the column, one can merge based on the index. This will only be appropriate if both indices are of the same type.

In [ ]:
sched_weather = pd.merge(schedule, weather, on = 'match_id')

In [ ]:
sched_weather

In [ ]:
sched_weather.temperature.describe()

In [ ]:
sched_weather.groupby('venue').temperature.describe()

In [ ]:
sched_weather.groupby('venue')[['temperature', 'wind_speed']].describe()

The other pandas functions for joining datasets that you should be aware of is `pd.concat`. This is appropriate when you are adding new rows to a dataframe (ingore the index) or new columns (may need to ignore index).

<a id='ref'></a>
# 3. References and Links
<a href=#top>(back to top)</a>

1. [Numpy user manual](https://numpy.org/doc/stable/user/index.html)
2. [10-minutes to Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/10min.html) User manuals and references can be found through here.
3. [Selection with Pandas](https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html) Using .loc and .iloc.
4. [World Happiness Report](https://worldhappiness.report/ed/2017/)